<h1>Media Appearances</h1>

This code is fetching articles from a media monitoring service Mediaboard (https://mediaboard.com) via API. Uses pagination to fetch all the data within a certain timeframe. Selects only relevant data, classifies articles based on brand relevancy and sends those data to Google Sheets to be later used as a source within Looker Studio dashboards.

In [1]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import date

In [2]:
APIToken = '###'
GSheetsCred = '###'

In [3]:
# Country selection

country = "CZ" # SK, HU

In [4]:
if country == "CZ":
    country_id = "67650"
    worksheet_id = 0
    keywords = ['nesnězeno', 'nesnezeno']
elif country == "SK":
    country_id = "67649"
    worksheet_id = 1
    keywords = ['munch']
elif country == "HU":
    country_id = "67648"
    worksheet_id = 2
    keywords = ['munch']

In [5]:
# Function to fetch data from API
def get_posts(link):
    url = link
    headers = {
        'Authorization': APIToken,
        'Content-type': 'application/json; charset=utf-8'
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # If the request was successful, parse the response as JSON
        posts = response.json()
        return posts
    else:
        # If the request failed, print the status code
        print(f"Failed to fetch posts. Status code: {response.status_code}")
        return None
    
# Function to count keywords
def count_keywords(row, keywords):
    count = sum(row.lower().count(keyword) for keyword in keywords)
    return count

# Function to categorize types of articles (About us, Mentioning us) based on mentions
def categorize_count(row):
    if row['title_keyword_count'] > 0: 
        return 'About us'
    else:
        if row['perex_keyword_count'] > 1 and row['text_keyword_count'] > 2:
            return 'About us'
        else:
            return 'Mention'

In [6]:
# First time updating - load data from 01-01-2023 to today
today = date.today()
formatted_date = today.strftime("%Y-%m-%d")
upper_date = formatted_date
lower_date = "2023-01-01" #normálně "2023-01-01"

In [8]:
#Mediaboard API connection + pagination

next_url = f'https://api.mediaboard.com/feed/{country_id}/?lower_date={lower_date}&upper_date={upper_date}&category_type=1'

my_list = []

while next_url:
    print(next_url)
    posts = get_posts(next_url)
    
    next_url = posts['next_url']
    articles = posts['articles']
    
    df = json_normalize(articles, sep='_')
    print(len(df))
    columns_to_keep = ['news_source_name', 'title', 'published', 'reach', 'url', 'text', 'perex', 'AVE', 'GRP']
    filtered_df = df[columns_to_keep]
    my_list.append(filtered_df)
    
final_df = pd.concat(my_list, ignore_index=True)

https://api.mediaboard.com/feed/67650/?lower_date=2023-01-01&upper_date=2024-03-17&category_type=1
20
https://api.mediaboard.com/feed/next/Z895s/
20
https://api.mediaboard.com/feed/next/Z895s/
16
https://api.mediaboard.com/feed/next/Z895s/
20
https://api.mediaboard.com/feed/next/Z895s/
20
https://api.mediaboard.com/feed/next/Z895s/
20
https://api.mediaboard.com/feed/next/Z895s/
20
https://api.mediaboard.com/feed/next/Z895s/
20
https://api.mediaboard.com/feed/next/Z895s/
20
https://api.mediaboard.com/feed/next/Z895s/
20
https://api.mediaboard.com/feed/next/Z895s/
20
https://api.mediaboard.com/feed/next/Z895s/
20
https://api.mediaboard.com/feed/next/Z895s/
2


In [9]:
# Mentions counts - title, text and perex + article classification (About us, Mentioning us)

final_df['title_keyword_count'] = final_df['title'].apply(count_keywords, args=(keywords,))
final_df['perex_keyword_count'] = final_df['perex'].apply(count_keywords, args=(keywords,))
final_df['text_keyword_count'] = final_df['text'].apply(count_keywords, args=(keywords,))

final_df['Status'] = final_df.apply(categorize_count, axis=1)

In [12]:
# Use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(GSheetsCred, scope)
client = gspread.authorize(creds)

# Open the spreadsheet and the first sheet
sheet = client.open("MediaboardAPI").get_worksheet(worksheet_id)
sheet.update('A1', [['Publisher', 'Headline','Date', 'Reach', 'Link','Text','Perex','AVE','GRP','TitleKeywords','TextKeywords','PerexKeywords','MentionType']])


/var/folders/g4/1x4xl1_92cj02tdf3gqfw5qm0000gn/T/ipykernel_66561/1603637236.py:8: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', [['Publisher', 'Headline','Date', 'Reach', 'Link','Text','Perex','AVE','GRP','TitleKeywords','TextKeywords','PerexKeywords','MentionType']])


{'spreadsheetId': '19m61vn7V7pstGqXQuhA2hXv5fVDobMOtx2yggjhMBhM',
 'updatedRange': "'Czech Republic'!A1:M1",
 'updatedRows': 1,
 'updatedColumns': 13,
 'updatedCells': 13}

In [13]:
# Latest empty cell row
column_values = sheet.col_values(1)
latest_empty_cell_row = len(column_values) + 1

In [14]:
# Remove duplicates
print (f"Original {len(final_df)}")
duplicates_removed = final_df.drop_duplicates(subset=['news_source_name','title'])
print (f"After duplicates removed {len(duplicates_removed)}")

Original 238
After duplicates removed 232


In [15]:
# Convert DataFrame to list of lists and update the sheet
data_to_upload = duplicates_removed.values.tolist()
sheet.update(f'A{latest_empty_cell_row}', data_to_upload)

/var/folders/g4/1x4xl1_92cj02tdf3gqfw5qm0000gn/T/ipykernel_66561/654501386.py:3: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update(f'A{latest_empty_cell_row}', data_to_upload)


{'spreadsheetId': '19m61vn7V7pstGqXQuhA2hXv5fVDobMOtx2yggjhMBhM',
 'updatedRange': "'Czech Republic'!A2:M233",
 'updatedRows': 232,
 'updatedColumns': 13,
 'updatedCells': 3016}